In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from numpy import load
from scipy.signal import periodogram
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

from keras.metrics import RootMeanSquaredError
import scipy.stats as stats

In [1]:
# conda install pytorch -c pytorch

In [2]:
# 加載 npz 文件
npz_file = "data/PEMS08.npz"
data = np.load(npz_file)
distance_df = pd.read_csv("data/distance.csv")

In [3]:
print(data['data'].shape) # 時間段數, 車站數量或感測器數量, 特徵數量
print(distance_df.head())

(17856, 170, 3)
   from   to   cost
0     9  153  310.6
1   153   62  330.9
2    62  111  332.9
3   111   11  324.2
4    11   28  336.0


In [5]:
# 流量, 佔有率, 速度
# data[key][1, 1, :]

In [6]:
distance_df.shape

(295, 3)

In [41]:
num_nodes = 170

# 構建鄰接矩陣
def build_adjacency_matrix(distance_df, num_nodes):
    adj_matrix = np.zeros((num_nodes, num_nodes))
    for _, row in distance_df.iterrows():
        from_node = int(row['from'])
        to_node = int(row['to'])
        cost = row['cost']
        adj_matrix[from_node, to_node] = cost
        adj_matrix[to_node, from_node] = cost  # 無向圖
    # return torch.tensor(adj_matrix, dtype=torch.float32)
    return adj_matrix

In [45]:
Adjacency = build_adjacency_matrix(distance_df, num_nodes)
np.save('data/Adjacency.npy', Adjacency)

In [9]:
lst = data.files
print(data[lst[0]].shape)
print(data[lst[0]])

(17856, 170, 3)
[[[1.330e+02 6.030e-02 6.580e+01]
  [2.100e+02 5.890e-02 6.960e+01]
  [1.240e+02 3.580e-02 6.580e+01]
  ...
  [7.400e+01 2.131e-01 6.530e+01]
  [9.400e+01 2.260e-02 6.800e+01]
  [6.000e+00 3.100e-03 6.500e+01]]

 [[1.140e+02 5.320e-02 6.690e+01]
  [1.850e+02 5.500e-02 6.850e+01]
  [1.190e+02 3.390e-02 6.500e+01]
  ...
  [7.300e+01 1.469e-01 3.720e+01]
  [8.400e+01 1.890e-02 6.870e+01]
  [4.000e+00 1.800e-03 6.500e+01]]

 [[1.400e+02 6.220e-02 6.680e+01]
  [1.710e+02 4.660e-02 6.990e+01]
  [1.070e+02 3.360e-02 6.380e+01]
  ...
  [7.000e+01 5.860e-02 3.400e+01]
  [8.200e+01 2.200e-02 6.700e+01]
  [4.000e+00 2.100e-03 6.490e+01]]

 ...

 [[1.200e+02 5.810e-02 6.330e+01]
  [1.760e+02 5.290e-02 6.680e+01]
  [1.190e+02 5.180e-02 5.610e+01]
  ...
  [4.700e+01 1.551e-01 3.220e+01]
  [9.100e+01 2.290e-02 6.640e+01]
  [3.000e+00 1.400e-03 6.530e+01]]

 [[1.020e+02 5.790e-02 6.140e+01]
  [1.650e+02 4.920e-02 6.720e+01]
  [1.330e+02 5.070e-02 5.890e+01]
  ...
  [9.700e+01 1.265e-01

The file gives a numpy array with dimensions (17856, 170, 3), which are in the form of (timesteps, location, features). For the 3 features, the order is flow,occupy,speed.

In [10]:
traffic_data = data[lst[0]]
data_dict = []
# loop for every timestep and every location and add as a single row
for timestep in range(traffic_data.shape[0]):
    for location in range(traffic_data.shape[1]):
        data_dict.append({
            "timestep" : timestep+1,
            "location" : location,
            "flow"     : traffic_data[timestep][location][0],
            "occupy"   : traffic_data[timestep][location][1],
            "speed"    : traffic_data[timestep][location][2]
        })

In [15]:
# df = pd.DataFrame(data_dict)
# df.to_csv("data/traffic.csv", index=False)

In [33]:
traffic = pd.read_csv("data/traffic.csv")
traffic

,timestep,location,flow,occupy,speed
0,1,0,133.0,0.0603,65.8
1,1,1,210.0,0.0589,69.6
2,1,2,124.0,0.0358,65.8
3,1,3,145.0,0.0416,69.6
4,1,4,206.0,0.0493,69.4
...,...,...,...,...,...
3035515,17856,165,74.0,0.0233,68.9
3035516,17856,166,11.0,0.0082,64.0
3035517,17856,167,83.0,0.0273,59.1
3035518,17856,168,70.0,0.0188,66.6


In [36]:
Adjacency

tensor([[  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000, 371.6000,  ...,   0.0000,   0.0000,   0.0000],
        [  0.0000, 371.6000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
        ...,
        [  0.0000,   0.0000,   0.0000,  ...,   0.0000, 172.4000,   0.0000],
        [  0.0000,   0.0000,   0.0000,  ..., 172.4000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000]])

In [23]:
# creating 3-dimensional array for [timestep, timeframe, features]
def create_dataset(location, WINDOW_SIZE):
    
    # mask a certain location
    location_current = traffic[traffic["location"]==location].reset_index()
    
    # group to hour and average 12 (5-minute) timesteps
    location_current["hour"] = ((location_current["timestep"] - 1) // 12)
    grouped = location_current.groupby("hour").mean().reset_index()
    
    # add hour features as mod 24 cycle (0...23)
    grouped['day'] = (grouped['hour'] // 24) % 7
    grouped['hour'] %= 24
    
    one_hot_hour = pd.get_dummies(grouped['hour'])
    one_hot_hour = one_hot_hour.add_prefix('hour_')
    
    # merge all the features together to get a total of 27 features
    hour_grouped = pd.concat([grouped[["occupy", "flow", "speed"]], one_hot_hour], axis=1)
    hour_grouped = np.array(hour_grouped)
    
    X, Y = [], []
    
    # add lag features (in reverse time order)
    for i in range(len(hour_grouped) - WINDOW_SIZE):
        X.append(hour_grouped[i:(i + WINDOW_SIZE)][::-1]) # reverse the order
        Y.append(hour_grouped[i + WINDOW_SIZE, 0]) # index 0 is occupy
    
    return X,Y # returns (timestep, timeframe, features) and (target)

In [24]:
# creating 4-th dimension for the locations
X, Y = [], []

for location in range(170):
    a,b = create_dataset(location, WINDOW_SIZE=24)
    X.append(a)
    Y.append(b)
    
X = np.moveaxis(X,0,-1)
Y = np.moveaxis(Y,0,-1)

print(X.shape)
print(Y.shape)

(1464, 24, 27, 170)
(1464, 170)


In [25]:
TRAIN_SIZE = 0.8
TEST_SIZE  = 0.2

train_size = int(len(X) * TRAIN_SIZE)
test_size  = int(len(X) * TEST_SIZE)

train_X, train_Y = X[:train_size], Y[:train_size]
test_X, test_Y = X[train_size:], Y[train_size:]

print(train_X.shape)
print(train_Y.shape)
print(test_X.shape)
print(test_Y.shape)

(1171, 24, 27, 170)
(1171, 170)
(293, 24, 27, 170)
(293, 170)


In [28]:
scaler_X = MinMaxScaler()
scaler_Y = MinMaxScaler()
train_X = scaler_X.fit_transform(train_X.reshape(train_X.shape[0] * train_X.shape[1], -1)) \
                   .reshape(train_X.shape[0], train_X.shape[1], -1)
test_X = scaler_X.transform(test_X.reshape(test_X.shape[0] * test_X.shape[1], -1)) \
                   .reshape(test_X.shape[0], test_X.shape[1], -1)
train_Y = scaler_Y.fit_transform(train_Y)
test_Y = scaler_Y.transform(test_Y)

In [29]:
model = Sequential([
    LSTM(256, return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2])),
    LSTM(256, return_sequences=False),
    Dropout(0.2),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(170, activation='linear'),
])

model.compile(loss='mse', optimizer='adam', metrics=[RootMeanSquaredError()])

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# from keras.utils.vis_utils import plot_model
# plot_model(model, show_shapes=True, show_layer_names=True)

In [31]:
# # Train the model
# history = model.fit(train_X, train_Y, epochs=150, batch_size=32, validation_split=0.1, verbose=2)